# Model Deployment to AWS SageMaker

This notebook outlines the steps to deploy the selected regression model to AWS SageMaker and includes code for making predictions.

## Step 1: Import Necessary Libraries


In [ ]:
import boto3
import pandas as pd
import joblib
from sagemaker import Session
from sagemaker.model import Model
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer


## Step 2: Load the Trained Model


In [ ]:
model_path = 's3://your-bucket-name/path/to/model/model.joblib'
model = joblib.load(model_path)


## Step 3: Define SageMaker Model


In [ ]:
sagemaker_session = Session()
role = 'your-sagemaker-execution-role'
model = Model(
    model_data=model_path,
    role=role,
    sagemaker_session=sagemaker_session,
    entry_point='inference.py',  # Your inference script
    framework_version='0.23-1',  # Adjust based on your model
    py_version='py3'
)


## Step 4: Deploy the Model


In [ ]:
predictor = model.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.large'
)

## Step 5: Make Predictions


In [ ]:
test_data = pd.read_csv('data/test.csv')
predictions = predictor.predict(test_data.values, initial_args={'ContentType': 'text/csv'})
predictions_df = pd.DataFrame(predictions, columns=['Predicted Price'])


## Step 6: Upload Predictions Back to Snowflake


In [ ]:
from scripts.upload_predictions import upload_to_snowflake
upload_to_snowflake(predictions_df)

## Step 7: Clean Up


In [ ]:
predictor.delete_endpoint()
